<a href="https://colab.research.google.com/github/MaJu502/proyectoDL/blob/main/modelo_PyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Proyecto Deep Learning
- Diego Cordova 20212
- Marco Jurado 20308
- Cristian Aguirre 20231
- Paola Contreras 20213
- Paola de Leon 20361

## Carga de Dataset

In [1]:
!gdown 1BoNWFSXxtN74PgLcBxfGWyeKUOfevQEP

Downloading...
From: https://drive.google.com/uc?id=1BoNWFSXxtN74PgLcBxfGWyeKUOfevQEP
To: /content/kaggle.json
100% 64.0/64.0 [00:00<00:00, 263kB/s]


In [2]:
! pip install kaggle

In [3]:
!ls -a /root/

.   .bashrc  .config   .jupyter  .keras		.local	.nv	  .tmux.conf
..  .cache   .ipython  .kaggle	 .launchpadlib	.npm	.profile  .wget-hsts


In [4]:
!mkdir /root/.kaggle/

mkdir: cannot create directory ‘/root/.kaggle/’: File exists


In [5]:
!mv /content/kaggle.json /root/.kaggle/


In [6]:
ls /root/.kaggle/


kaggle.json


In [7]:
!chmod 600 /root/.kaggle/kaggle.json

In [8]:
!kaggle datasets download -d jonathanoheix/face-expression-recognition-dataset

face-expression-recognition-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [9]:
%cd /content
!unzip face-expression-recognition-dataset.zip

/content
Archive:  face-expression-recognition-dataset.zip
replace images/images/train/angry/0.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

# Modelo

In [10]:
import cv2
import os
import numpy as np

# Directorio raíz que contiene subdirectorios para cada emoción
root_dir = './images/train/'

# Lista de emociones
emotions = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']

# Inicializa un diccionario para almacenar shapes únicos por emoción
unique_shapes_by_emotion = {emotion: [] for emotion in emotions}

# Itera a través de las carpetas de emociones
for emotion in emotions:
    emotion_dir = os.path.join(root_dir, emotion)
    image_files = os.listdir(emotion_dir)

    # Itera a través de los archivos de imágenes en cada carpeta de emoción
    for image_file in image_files:
        image_path = os.path.join(emotion_dir, image_file)
        image = cv2.imread(image_path)
        shape = image.shape
        if shape not in unique_shapes_by_emotion[emotion]:
            unique_shapes_by_emotion[emotion].append(shape)

# Imprime los shapes únicos por emoción
for emotion, shapes in unique_shapes_by_emotion.items():
    print(f"Shapes únicos para la emoción '{emotion}':")
    for shape in shapes:
        print(shape)


Shapes únicos para la emoción 'angry':
(48, 48, 3)
Shapes únicos para la emoción 'disgust':
(48, 48, 3)
Shapes únicos para la emoción 'fear':
(48, 48, 3)
Shapes únicos para la emoción 'happy':
(48, 48, 3)
Shapes únicos para la emoción 'neutral':
(48, 48, 3)
Shapes únicos para la emoción 'sad':
(48, 48, 3)
Shapes únicos para la emoción 'surprise':
(48, 48, 3)


## Loader para el Modelo

In [11]:
import torch
from torchvision import datasets, transforms

batch_size = 128
base_path = "./images/"

# Definir transformaciones para el conjunto de entrenamiento y validación
train_transform = transforms.Compose([
    transforms.Resize((56, 56)),
    transforms.Grayscale(num_output_channels=1),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(20),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])  # Normalizar imágenes
])

validation_transform = transforms.Compose([
    transforms.Resize((56, 56)),
    transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])  # Normalizar imágenes
])

# Crear conjuntos de datos
train_dataset = datasets.ImageFolder(base_path + "train", transform=train_transform)
validation_dataset = datasets.ImageFolder(base_path + "validation", transform=validation_transform)

# Crear generadores de lotes (DataLoaders)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
validation_loader = torch.utils.data.DataLoader(validation_dataset, batch_size=batch_size, shuffle=False)
for i, data in enumerate(train_loader, 0):
  print(np.array(data[0]).shape)
  break

(128, 1, 56, 56)


In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.nn.modules import activation

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

class Sentiment(nn.Module):
    def __init__(self, num_classes=10):
        super(Sentiment, self).__init__()

        self.convolucionales = nn.Sequential(
            # Capa convolucional 1
            # - 64 kernels de 3x3
            # - activacion ReLU
            # - MaxPooling con kernel de 2x2
            # - Dropout de 0.25
            nn.Conv2d(in_channels=1, out_channels=64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Dropout(0.25),

            # Capa convolucional 2
            # - 128 kernels de 5x5
            # - activacion ReLU
            # - MaxPooling con kernel de 2x2
            # - Dropout de 0.25
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=5, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Dropout(0.25),

            # Capa convolucional 3
            # - 512 kernels de 3x3
            # - activacion ReLU
            # - MaxPooling con kernel de 2x2
            # - Dropout de 0.25
            nn.Conv2d(in_channels=128, out_channels=512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Dropout(0.25),

            # Capa convolucional 4 (igual a la 3)
            # - 512 kernels de 3x3
            # - activacion ReLU
            # - MaxPooling con kernel de 2x2
            # - Dropout de 0.25
            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Dropout(0.25),
        )

        # Calcular el tamaño de la salida de las capas convolucionales
        self.fc_input_size = self._get_conv_output_size((128, 1, 56, 56))

        # Fully Connected
        self.classifier = nn.Sequential(
          # Se aplanan las salidas convolucionales
          nn.Flatten(),

          # Fully Connected 1
          # - activacion ReLU
          # - Dropout de 0.25
          # nn.Linear(64 * 28 * 28, 256),
          nn.Linear(4608, 256),
          nn.BatchNorm1d(256),
          nn.ReLU(),
          nn.Dropout(0.25),

          # Fully Connected 2
          # - activacion ReLU
          # - Dropout de 0.25
          nn.Linear(256, 512),
          nn.BatchNorm1d(512),
          nn.ReLU(),
          nn.Dropout(0.25),

          # Fully Connected 3 (salida)
          # - activacion spftmax
          nn.Linear(512, num_classes),
          nn.Softmax(dim=1),
        )

    def _get_conv_output_size(self, shape):
        x = torch.rand(shape)
        x = self.convolucionales(x)
        return x.view(1, -1).size(1)

    def forward(self, x):
        x = self.convolucionales(x)
        x = self.classifier(x)
        return x

modelo = Sentiment(len(emotions))
print(modelo)

cuda:0
Sentiment(
  (convolucionales): Sequential(
    (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Dropout(p=0.25, inplace=False)
    (5): Conv2d(64, 128, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
    (6): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): ReLU()
    (8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (9): Dropout(p=0.25, inplace=False)
    (10): Conv2d(128, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU()
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Dropout(p=0.25, inplace=False)
    (15): Conv2d(512, 512, k

## Se preparan los datos para CUDA

In [13]:
device = ('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Computation device: {device}\n")
modelo = modelo.to(device)

device_in_model = 'cpu' if False in [i.is_cuda for i in modelo.parameters()] else 'cuda'
print('model using', device_in_model)

Computation device: cuda

model using cuda


## Entreno del Modelo

In [ ]:
import torch.optim as optim

# Función de pérdida y optimizador
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(modelo.parameters(), lr=0.001, momentum=0.9)

epochs = 30

# Entrenamiento
for epoch in range(epochs):
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = modelo(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    # Evaluación en el conjunto de prueba
    correct = 0
    total = 0
    with torch.no_grad():
        for data in validation_loader:
            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = modelo(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f'Epoch {epoch+1} | Loss: {running_loss / len(train_loader)} | Accuracy: {100 * correct / total}%')

print('Finished Training')

In [ ]:
import pickle

model_filename = "model.pkl"

with open(model_filename, 'wb') as model_file:
    pickle.dump(model, model_file)


In [ ]:
import matplotlib.pyplot as plt

def plot_results(history):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    epochs = range(1, len(acc) + 1)

    plt.figure(figsize = (24, 6))
    plt.subplot(1,2,1)
    plt.plot(epochs, acc, 'b', label = 'Training Accuracy')
    plt.plot(epochs, val_acc, 'r', label = 'Validation Accuracy')
    plt.grid(True)
    plt.legend()
    plt.xlabel('Epoch')



    plt.subplot(1,2,2)
    plt.plot(epochs, loss, 'b', label = 'Training Loss')
    plt.plot(epochs, val_loss, 'r', label = 'Validation Loss')
    plt.grid(True)
    plt.legend()
    plt.xlabel('Epoch')
    plt.show()

def get_best_epcoh(history):
    valid_acc = history.history['val_accuracy']
    best_epoch = valid_acc.index(max(valid_acc)) + 1
    best_acc =  max(valid_acc)
    print('Best Validation Accuracy Score {:0.5f}, is for epoch {}'.format( best_acc, best_epoch))
    return best_epoch

In [ ]:
plot_results(history)
best_epoch =get_best_epcoh(history)